In [1]:
from utils import process_tweet, lookup  # For pre processing of tweets
import pdb
import math
import nltk
import string
import re   # Regular expression
from nltk.tokenize import TweetTokenizer
import csv # to read the csv file of stop words
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import sklearn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

In [85]:
print("SKLEARN:",sklearn.__version__)
print("NUMPY:",np.__version__)
print("NLTK:",nltk.__version__)
import scipy
print("SCIPY",scipy.__version__)

req = '''pip install -U scikit-learn==0.19.1
pip install numpy==1.14.3
pip install nltk==3.3
'''
print(req)

SKLEARN: 0.19.1
NUMPY: 1.14.3
NLTK: 3.3
SCIPY 1.1.0
pip install -U scikit-learn==0.19.1
pip install numpy==1.14.3
pip install nltk==3.3



In [66]:
import csv
Stop_Words=[]
with open('StopWords.csv', 'r') as file:  # Read the stop words from file and store them in the List
    reader = csv.reader(file)
    i=0
    for row in reader:
        Stop_Words.insert(i,row)
        i+=1
file.close()

In [67]:
# A function to pre process the tweets
def Pre_Process(Tweets):
    
    # remove stock market tickers like $GE

    #Tweets = re.sub(r'\$\w*', '', Tweets)

    # remove old style retweet text "RT"

     #  tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks

 #   tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags

    # only removing the hash # sign from the word

  #  tweet = re.sub(r'#', '', tweet)
    
    # tokenize tweets
    Tokenized_Tweets=[[]]
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True) # Case-Folding, Tokenization
    for i in range(len(Tweets)-1):
        tweet_tokens = tokenizer.tokenize(Tweets[i])
        Tokenized_Tweets.insert(i,tweet_tokens)
    #print("Number of Tokenized Tweets = "+str(len(Tokenized_Tweets)))
    #print(Tokenized_Tweets)
    #print("len(Tokenized)",len(Tokenized_Tweets))
    cleaned_tweets = []
    for i in range (len(Tokenized_Tweets)-1): # For all tweets
        inner=[]
        for k in range(0,len(Tokenized_Tweets[i])): # For all the words of one tweet
            check=0
            for j in range(len(Stop_Words)):        # Iterate for all the words of one tweet in Stop Words List
                if(Tokenized_Tweets[i][k]==Stop_Words[j][0]): #check if the words of one tweet are in Stop Words List
                    check=1 
                    break
            if Tokenized_Tweets[i][k] in string.punctuation:  # Removes punctuation
                check=1
            if not check:        
                inner.append(Tokenized_Tweets[i][k])
        cleaned_tweets.append(inner)
    return cleaned_tweets
    
    

In [68]:
# convert the tokenized tweets back to string
def Remove_Stop_Words(x):# Combine tokenzied words to strings again after removing stop words

    Tweets=[]
    b=''
    i=0
    for tweet in x:
        for word in tweet:
            b=b+word
            b=b+' '
        Tweets.insert(i,b)
        i+=1
    return Tweets

In [69]:
# Load the data  of tweets

import xlrd       # For reading excel file
from array import *   
loc=("C:\\Users\\Mr.Wick\\Videos\\Fall2020\\FYP\\dataset.xlsx") # Loading dataset 
wb = xlrd.open_workbook(loc)   
sheet = wb.sheet_by_index(0)



TWEETS=[]   # a list to store tweets
labels=[]   # A list to store labels

rows=sheet.nrows

for i in range(1,rows):
    labels.insert(i,sheet.cell_value(i,1))
    TWEETS.insert(i,sheet.cell_value(i,0))


In [70]:
cleaned_tweets=Pre_Process(TWEETS)
t= Remove_Stop_Words(cleaned_tweets)
print(t[1])

karkhanon dhuwan mahol aloodah kerta lahore mazeed 7 elaaqay seal ker diye gaye 


In [72]:
#Converting the tweets to the TF-IDFs
tfidfconverter = TfidfVectorizer()
X = tfidfconverter.fit(t)
print(X)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


NameError: name 'tfidf' is not defined

In [75]:
pickle.dump(X, open("tfidf.pickle", "wb"))

In [ ]:
X = tfidfconverter.transform(X).toarray()
print(len(X))

In [26]:
print(X.shape)

(4999, 5602)


In [15]:
# Splitting the data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)

In [16]:
# Neural Network 
classifier3=MLPClassifier(alpha=1, max_iter=2000)
classifier3.fit(X_train, y_train)
Y_pred = classifier3.predict(X_test)
print(confusion_matrix(y_test,Y_pred))
print(classification_report(y_test,Y_pred))

print(accuracy_score(y_test, Y_pred))

[[567  35]
 [  0 398]]
             precision    recall  f1-score   support

          N       1.00      0.94      0.97       602
          R       0.92      1.00      0.96       398

avg / total       0.97      0.96      0.97      1000

0.965


In [77]:
import numpy as np
a=['log kehtey hein ke taqreer se kia hoga, hazrat jafer tayyar ne bhi taqreer ki thi, jis ne najashi ka dil badal diya tha',[]]
print(len(a))
tfidfconverter = pickle.load(open('tfidf.pickle', 'rb'))
a=Pre_Process(a)
print("After Pre_Processing", a)
a=Remove_Stop_Words(a)
print("Converting back to string",a)
x = tfidfconverter.transform(a).toarray()
print("tf-idf",x)
y_pred = classifier3.predict(x)
print(y_pred)

2
After Pre_Processing [['log', 'kehtey', 'taqreer', 'kia', 'hoga', 'hazrat', 'jafer', 'tayyar', 'taqreer', 'jis', 'najashi', 'dil', 'badal', 'diya']]
Converting back to string ['log kehtey taqreer kia hoga hazrat jafer tayyar taqreer jis najashi dil badal diya ']
tf-idf [[0. 0. 0. ... 0. 0. 0.]]
['R']


In [53]:
#Save neural network model
import pickle
filename='NeuralNetworks.sav'
pickle.dump(classifier3, open(filename, 'wb'))

In [54]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.965


In [78]:
print(sklearn.__version__)

0.19.1
